## 모듈

1. et
2. lgb
3. rf
4. xgb hist
5. knn
6. + cb
7. + xgb

In [1]:
!pip3 install optuna
!pip3 install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import datetime
import pickle
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir('/content/drive/MyDrive/')
import warnings
warnings.filterwarnings('ignore')
from tqdm import tqdm
import itertools


import pandas as pd
import pandas_profiling
import numpy as np
import tensorflow as tf


import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme(style='darkgrid' )
import plotly.offline as pyo
import plotly.express as px
import plotly.graph_objects as go
pyo.init_notebook_mode()
#import scikitplot

from scipy import stats


import sklearn
from sklearn import preprocessing
from sklearn import decomposition
from sklearn import metrics
from sklearn.utils.fixes import loguniform
from sklearn import model_selection
from sklearn import ensemble
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import ExtraTreesRegressor


import optuna
from optuna.samplers import TPESampler
import xgboost as xgb
import lightgbm as lgb 
import xgboost as xgb
import catboost as cb
import statsmodels.api as sm

Output hidden; open in https://colab.research.google.com to view.

## load data

In [3]:
with open("/content/drive/MyDrive/날씨/lgb_train.pkl","rb") as fr:
    data = pickle.load(fr)
df = data.copy()

In [4]:
kfcv = model_selection.KFold(10,shuffle=True,random_state=42)

### 함수

In [5]:
def  stn_split(df:pd.DataFrame,spn_num :int)->pd.DataFrame:
  if 'stn' in df.columns:
    df_stn_num = df.loc[df['stn']==spn_num,:]
    df_stn_num = df_stn_num.drop(['stn'],axis=1)
    return df_stn_num  

def tr_ts_split(df:pd.DataFrame)->(pd.DataFrame,pd.DataFrame):
  train_df = df.loc[(df.index<'2021-06')| (df.index>='2021-07-01'),:]
  test_df = df.loc[(df.index>='2021-06')&(df.index<'2021-07'),:]
  return train_df,test_df


def stationarity_test(df:pd.Series)-> (pd.DataFrame,pd.DataFrame):
  Stationarity_kpss = sm.tsa.stattools.kpss(df)
  Stationarity_kpss_df = pd.Series(Stationarity_kpss[0:3],
                                   index=['Test Statistics', 'p-value', 'Used Lag'])
  for key, value in Stationarity_kpss[3].items():
    Stationarity_kpss_df[f'Critical Value {key}'] = value
  Stationarity_kpss_df = pd.DataFrame(Stationarity_kpss_df, columns=['Stationarity_kpss'])
  
  Stationarity_adf = sm.tsa.stattools.adfuller(df)
  Stationarity_adf_df = pd.Series(Stationarity_adf[0:4],
                                  index=['Test Statistics', 'p-value', 'Used Lag', 'Used Observations'])
  for key, value in Stationarity_adf[4].items():
    Stationarity_adf_df[f'Critical Value {key}'] = value
  Stationarity_adf_df['Maximum Information Criteria'] = Stationarity_adf[5]
  Stationarity_adf_df = pd.DataFrame(Stationarity_adf_df, columns=['Stationarity_adf'])

  return pd.merge(Stationarity_kpss_df, Stationarity_adf_df, left_index=True,  right_index = True , how ='outer')


def time_feature(df:pd.DataFrame)->pd.DataFrame:
  x_list = ['solarza','esr','band1','band2','band3','band4','band6','band7','band12']
  df_feature = df.copy()
  for x in x_list:
    for n in [1,144]:
      if f'{x}_{n}' not in df_feature.columns:
        df_feature[f'{x}_{n}'] = df_feature[x].diff(n).bfill()
    for n in [72,144]:
      if f'{x}_{n}ma' not in df_feature.columns:
        df_feature[f'{x}_{n}ma'] = df_feature[x].rolling(n).mean().bfill()
  return df_feature

## 튜닝

In [6]:
df_146 = stn_split(df,146)

In [7]:
df_146 = time_feature(df_146)
df_146.columns

Index(['uv', 'band1', 'band2', 'band3', 'band4', 'band5', 'band6', 'band7',
       'band8', 'band9', 'band10', 'band11', 'band12', 'band13', 'band14',
       'band15', 'band16', 'solarza', 'esr', 'year', 'Month sin', 'Month cos',
       'Day sin', 'Day cos', '2/Day sin', '2/Day cos', '3/Day sin',
       '3/Day cos', 'solarza_1', 'solarza_144', 'solarza_72ma',
       'solarza_144ma', 'esr_1', 'esr_144', 'esr_72ma', 'esr_144ma', 'band1_1',
       'band1_144', 'band1_72ma', 'band1_144ma', 'band2_1', 'band2_144',
       'band2_72ma', 'band2_144ma', 'band3_1', 'band3_144', 'band3_72ma',
       'band3_144ma', 'band4_1', 'band4_144', 'band4_72ma', 'band4_144ma',
       'band6_1', 'band6_144', 'band6_72ma', 'band6_144ma', 'band7_1',
       'band7_144', 'band7_72ma', 'band7_144ma', 'band12_1', 'band12_144',
       'band12_72ma', 'band12_144ma'],
      dtype='object')

In [8]:
df_146.index

DatetimeIndex(['2020-01-01 00:00:00', '2020-01-01 00:10:00',
               '2020-01-01 00:20:00', '2020-01-01 00:30:00',
               '2020-01-01 00:40:00', '2020-01-01 00:50:00',
               '2020-01-01 01:00:00', '2020-01-01 01:10:00',
               '2020-01-01 01:20:00', '2020-01-01 01:30:00',
               ...
               '2021-12-31 22:20:00', '2021-12-31 22:30:00',
               '2021-12-31 22:40:00', '2021-12-31 22:50:00',
               '2021-12-31 23:00:00', '2021-12-31 23:10:00',
               '2021-12-31 23:20:00', '2021-12-31 23:30:00',
               '2021-12-31 23:40:00', '2021-12-31 23:50:00'],
              dtype='datetime64[ns]', name='date_time', length=105264, freq=None)

In [15]:
train,test = tr_ts_split(df_146)

In [10]:
df_146.shape[0] == train.shape[0] + test.shape[0]

True

### ET

In [ ]:
etr = ExtraTreesRegressor(random_state=42,n_jobs=-1)

In [ ]:
etr_rand_params = {'min_weight_fraction_leaf':[0.0, 0.  , 0.005, 0.0075, 0.01, 0.05],
                  'min_samples_split':[2, 0.01, 0.02, 0.03, 0.04, 0.06, 0.08, 0.1],
                  'min_samples_leaf':[1,2,4,6,8,10,20,30],
                  'min_impurity_decrease':[0.0, 0.01, 0.05, 0.10, 0.15, 0.2],
                  'max_leaf_nodes':[10, 15, 20, 25, 30, 35, 40, 45, 50, None],
                  'max_features':['sqrt', 0.8, 0.7, 0.6, 0.5, 0.4],
                  'max_depth':[None,2,4,6,8,10,20],
				      		'n_estimators':[100,200,300]}

In [ ]:
etr_rf = model_selection.RandomizedSearchCV(etr ,
                                              etr_rand_params,
                                              n_iter=200,
                                              cv=kfcv,
                                              random_state=42,
                                              scoring ='neg_root_mean_squared_error',
                                              return_train_score=True,
                                              n_jobs=-1)
etr_rf.fit(train_108.iloc[:,1:],train_108['uv'])

RandomizedSearchCV(cv=KFold(n_splits=10, random_state=42, shuffle=True),
                   estimator=ExtraTreesRegressor(n_jobs=-1, random_state=42),
                   n_iter=200, n_jobs=-1,
                   param_distributions={'max_depth': [None, 2, 4, 6, 8, 10, 20],
                                        'max_features': ['sqrt', 0.8, 0.7, 0.6,
                                                         0.5, 0.4],
                                        'max_leaf_nodes': [10, 15, 20, 25, 30,
                                                           35, 40, 45, 50,
                                                           None],
                                        'min_impurity_decrease': [0.0, 0.01,
                                                                  0.05, 0.1,
                                                                  0.15, 0.2],
                                        'min_samples_leaf': [1, 2, 4, 6, 8, 10,
                                              

In [ ]:
etr_rf.best_params_

{'max_depth': 8,
 'max_features': 0.7,
 'max_leaf_nodes': 40,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 2,
 'min_samples_split': 0.01,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 300}

In [ ]:
etr_rf.best_score_

-0.3863772394741232

In [ ]:
X = df_108.iloc[:,1:]
y = df_108['uv']
_n_trails = 100

def tune(objective):
    sampler = TPESampler(seed=42)
    study = optuna.create_study(direction="maximize",sampler = sampler)
    study.optimize(objective, n_trials=_n_trails)
    params = study.best_params
    best_score = study.best_value
    print(f"Best score: {best_score}\n")
    print(f"Optimized parameters: {params}\n")
    return params

def etr_objective(trial):
    etr_param = {
          'n_estimators' : trial.suggest_int('n_estimators',200,600),
          'max_depth' : trial.suggest_int('max_depth',15,35),
          'min_weight_fraction_leaf' : trial.suggest_float('min_weight_fraction_leaf',0.0,0.001),
          'min_samples_split' : trial.suggest_float('min_samples_split',0.0,0.02),
          'min_samples_leaf' : trial.suggest_int('min_samples_leaf',1,5),
          'max_features' : trial.suggest_float('max_features',0.5,0.9),
          'min_impurity_decrease':trial.suggest_float('min_impurity_decrease',0.0,0.001)
    }



    etr = ExtraTreesRegressor(**etr_param,random_state=42,n_jobs=-1)
    
    
    scores = model_selection.cross_val_score(etr, X, y, cv=kfcv, 
                             scoring="neg_root_mean_squared_error"
                             )
    return scores.mean()

In [ ]:
etr_best_params = tune(etr_objective)

[I 2022-07-30 20:11:47,630] A new study created in memory with name: no-name-9798de38-b543-43c0-83df-46ee980dfdc1
[I 2022-07-30 20:16:08,774] Trial 0 finished with value: -0.3929654548296809 and parameters: {'n_estimators': 350, 'max_depth': 34, 'min_weight_fraction_leaf': 0.0007319939418114051, 'min_samples_split': 0.011973169683940733, 'min_samples_leaf': 1, 'max_features': 0.562397808134481, 'min_impurity_decrease': 5.808361216819946e-05}. Best is trial 0 with value: -0.3929654548296809.
[I 2022-07-30 20:26:30,028] Trial 1 finished with value: -0.3498474783050062 and parameters: {'n_estimators': 547, 'max_depth': 27, 'min_weight_fraction_leaf': 0.0007080725777960455, 'min_samples_split': 0.00041168988591604893, 'min_samples_leaf': 5, 'max_features': 0.8329770563201687, 'min_impurity_decrease': 0.00021233911067827617}. Best is trial 1 with value: -0.3498474783050062.
[I 2022-07-30 20:29:47,695] Trial 2 finished with value: -0.38866438983761914 and parameters: {'n_estimators': 272, 'm

Best score: -0.3029884569089051

Optimized parameters: {'n_estimators': 398, 'max_depth': 25, 'min_weight_fraction_leaf': 7.412742668612509e-05, 'min_samples_split': 0.00035665374622706513, 'min_samples_leaf': 1, 'max_features': 0.8203707539398322, 'min_impurity_decrease': 2.6521473059677293e-05}



### CABOOST

In [9]:
df_146 = df_146.loc[df_146.index.month.isin([5,6,7,8]),:]
kfcv = model_selection.KFold(n_splits = 3 )

In [17]:
#GPU
X = df_146.iloc[:,1:]
y = df_146['uv']
_n_trials = 100

def tune(objective):
    _sampler = optuna.samplers.TPESampler(seed=42)
    study = optuna.create_study(direction="maximize",
                                sampler = _sampler
                                )
    study.optimize(objective, n_trials=_n_trials)
    params = study.best_params
    best_score = study.best_value
    print(f"Best score: {best_score}\n")
    print(f"Optimized parameters: {params}\n")
    return params,study



def cb_objective(trial):
  cb_param = {
  'grow_policy' : trial.suggest_categorical('grow_policy',[
#                                                           'SymmetricTree', 
#                                                           'Depthwise',
                                                            'Lossguide']),
#  'boosting_type' : trial.suggest_categorical("boosting_type", ["Ordered", "Plain"]), # GPU 멀티클래스 만 가능
  'learning_rate' : trial.suggest_loguniform('learning_rate', 0.01, 0.8),
  'n_estimators' : trial.suggest_int("n_estimators", 300,5000),
  'max_depth' : trial.suggest_int("max_depth", 2, 16),  #Lossguide만 제외하고 MAX 16
  'random_strength' :trial.suggest_int('random_strength', 0, 100), #CPU 만 지원
  'l2_leaf_reg' : trial.suggest_loguniform("l2_leaf_reg",1e-6,3.0),
  'max_bin' : trial.suggest_int("max_bin", 25, 300),
  'od_type' : trial.suggest_categorical('od_type', ['IncToDec', 'Iter']),
#  'colsample_bylevel' : trial.suggest_discrete_uniform('colsample_bylevel',0.5,1.0,0.05), #CPU 만 지원
  'bootstrap_type' :  trial.suggest_categorical("bootstrap_type", ["Bayesian", "Bernoulli", "Poisson"])} #GPU만 Poisson 가능
  
  if cb_param['grow_policy'] == 'Lossguide' or  cb_param['grow_policy'] == 'Depthwise':
    cb_param['min_child_samples'] = trial.suggest_int('min_child_samples',1,100)
  
  if cb_param['grow_policy'] == 'Lossguide':
    cb_param['num_leaves'] = trial.suggest_int('num_leaves',20,50)

  if cb_param['bootstrap_type'] =='Bayesian':
    cb_param['bagging_temperature'] = trial.suggest_loguniform('bagging_temperature', 0.01, 100.00)
  elif cb_param['bootstrap_type'] =='Bernoulli' or cb_param['bootstrap_type'] =='Poisson':
    cb_param['subsample'] = trial.suggest_discrete_uniform('subsample', 0.6, 1.0, 0.1)

  
  _fit_params={'early_stopping_rounds':100,
               'eval_set': [(X,y)],
              'verbose':0}

  cbr = cb.CatBoostRegressor(
                             random_state=42,
                             task_type = 'GPU',
                              **cb_param
                             )

    
  scores = model_selection.cross_val_score(cbr, X, y, cv=kfcv, 
                           scoring ="neg_root_mean_squared_error",
                           fit_params = _fit_params
                           )
                            
  return scores.mean()
  
cbr_params,cbr_study = tune(cb_objective)

[I 2022-08-01 16:33:25,074] A new study created in memory with name: no-name-9da1e85a-8ccd-47cb-9066-184feabdd0ee
[I 2022-08-01 16:35:21,948] Trial 0 finished with value: -0.8947434700778077 and parameters: {'grow_policy': 'Lossguide', 'learning_rate': 0.05161590541863461, 'n_estimators': 4769, 'max_depth': 12, 'random_strength': 60, 'l2_leaf_reg': 1.024593631522237e-05, 'max_bin': 68, 'od_type': 'Iter', 'bootstrap_type': 'Bernoulli', 'min_child_samples': 97, 'num_leaves': 45, 'subsample': 0.7}. Best is trial 0 with value: -0.8947434700778077.
[I 2022-08-01 16:35:53,724] Trial 1 finished with value: -0.8856652079923196 and parameters: {'grow_policy': 'Lossguide', 'learning_rate': 0.022183459467487607, 'n_estimators': 1162, 'max_depth': 6, 'random_strength': 53, 'l2_leaf_reg': 0.0006277117542293967, 'max_bin': 105, 'od_type': 'IncToDec', 'bootstrap_type': 'Poisson', 'min_child_samples': 79, 'num_leaves': 26, 'subsample': 0.8}. Best is trial 1 with value: -0.8856652079923196.
[I 2022-08-

KeyboardInterrupt: ignored

In [ ]:
#CPU
X = df_146.iloc[:,1:]
y = df_146['uv']
_n_trials = 300

def tune(objective):
    _sampler = optuna.samplers.TPESampler(seed=42)
    study = optuna.create_study(direction="maximize",
                                sampler = _sampler
                                )
    study.optimize(objective, n_trials=_n_trials)
    params = study.best_params
    best_score = study.best_value
    print(f"Best score: {best_score}\n")
    print(f"Optimized parameters: {params}\n")
    return params,study



def cb_objective(trial):
  cb_param = {
  'grow_policy' : trial.suggest_categorical('grow_policy',[
                                                           'SymmetricTree', 
                                                           'Depthwise',
                                                           'Lossguide']),
# 'boosting_type' : trial.suggest_categorical("boosting_type", ["Ordered", "Plain"]), # GPU 멀티클래스 만 가능
  'learning_rate' : trial.suggest_loguniform('learning_rate', 0.01, 0.8),
  'n_estimators' : trial.suggest_int("n_estimators", 300,5000),
  'max_depth' : trial.suggest_int("max_depth", 2, 16),  #Lossguide만 제외하고 MAX 16
  'random_strength' :trial.suggest_int('random_strength', 0, 100), #CPU 만 지원
  'l2_leaf_reg' : trial.suggest_loguniform("l2_leaf_reg",1e-6,3.0),
  'max_bin' : trial.suggest_int("max_bin", 25, 300),
  'od_type' : trial.suggest_categorical('od_type', ['IncToDec', 'Iter']),
  'colsample_bylevel' : trial.suggest_discrete_uniform('colsample_bylevel',0.5,1.0,0.05), #CPU 만 지원
  'bootstrap_type' :  trial.suggest_categorical("bootstrap_type", ["Bayesian", "Bernoulli", "MVS"]), #GPU만 Poisson 가능
  #'sampling_frequency': trial.suggest_categorical("sampling_frequency", ["PerTree", "PerTreeLevel"]),
  #'sampling_unit': trial.suggest_categorical("sampling_unit", [ "Group"])
   }

  if cb_param['grow_policy'] == 'Lossguide' or  cb_param['grow_policy'] == 'Depthwise':
    cb_param['min_child_samples'] = trial.suggest_int('min_child_samples',1,100)
  
  if cb_param['grow_policy'] == 'Lossguide':
    cb_param['num_leaves'] = trial.suggest_int('num_leaves',20,50)

  if cb_param['bootstrap_type'] =='Bayesian':
    cb_param['bagging_temperature'] = trial.suggest_loguniform('bagging_temperature', 0.01, 100.00)
  elif cb_param['bootstrap_type'] =='Bernoulli' or cb_param['bootstrap_type'] =='MVS':
    cb_param['subsample'] = trial.suggest_discrete_uniform('subsample', 0.6, 1.0, 0.1)

  
  _fit_params={'early_stopping_rounds':100,
               'eval_set': [(X,y)],
              'verbose':0}

  cbr = cb.CatBoostRegressor(
                             random_state=42,
 #                            task_type = 'GPU',
                              **cb_param
                             )

    
  scores = model_selection.cross_val_score(cbr, X, y, cv=kfcv, 
                           scoring ="neg_root_mean_squared_error",
                           fit_params = _fit_params
                           )
                            
  return scores.mean()
  
cbr_params,cbr_study = tune(cb_objective)

[I 2022-08-01 18:58:15,598] A new study created in memory with name: no-name-529f87e2-dbc9-42cf-aa3d-03a769cd3bb9


In [22]:
 cbr = cb.CatBoostRegressor(
                             random_state=42,
                             task_type = 'GPU',
                             bootstrap_type='Bayesian',
                             sampling_unit = "Group"
                             )

In [23]:
cbr.fit(X,y)

CatBoostError: ignored